# Construction Loan Estimator

## Parameters

In [1]:
# All imports
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from dateutil.relativedelta import relativedelta
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
from math import e

# Global options
pd.set_option('display.max_columns', None)
output_notebook()

Loading BokehJS ...

In [8]:
# Set scenario parameters

# Initial Costs
property_cost = 85000
construction_cost = 115000
down_payment = 40000
completed_value = 300000
closing_costs = 0.05

# Terms
loan_term = 15
construction_term = 12
years_to_consider = 30

# Ongoing costs
maintenance_per_sqft = 0.75
assessment_rate = 0.07
mill_levy = 95
initial_hoa = 150

# Rent costs
initial_rent = 1500

# Economic factors
inflation = 0.02
rent_growth_rate = 0.02
home_appreciation = 0.05
loan_interest_rate = 0.04
investment_growth_rate = 0.06

# Property estimates
square_footage = 1750

# Conversions
total_loan_amount = property_cost + construction_cost - down_payment
loan_rate_monthly = loan_interest_rate / 12
investment_growth_rate_monthly = investment_growth_rate / 12
rent_growth_rate_monthly = rent_growth_rate / 12
home_appreciation_month = home_appreciation / 12
current_month = datetime.date.today().replace(day=1)
loan_term_months = loan_term * 12
months_to_consider = years_to_consider * 12
inflation_monthly = inflation / 12

In [11]:
# Create blank table for monthly level data
monthly_data = pd.DataFrame()

# Create Period to offset the index by one, to indicate month and create periods
monthly_data['Overall Month'] = range(1, months_to_consider + 2)
monthly_data['Overall Year'] = ((monthly_data['Overall Month'] - 1) // 12)
monthly_data['Year Start'] = np.where((((monthly_data['Overall Month'] - 1) // 12) == (monthly_data['Overall Month'] - 1) / 12) | (monthly_data['Overall Month'] == 1), 1, 0)
monthly_data['Construction Month'] = np.where(monthly_data['Overall Month'] <= construction_term, monthly_data['Overall Month'], 0)
monthly_data['Loan Month'] = np.where((monthly_data['Overall Month'] > construction_term) & (monthly_data['Overall Month'] < (construction_term + loan_term_months)), monthly_data['Overall Month'] - construction_term, 0)

# Calculate home value
monthly_data['Home Value'] = np.where(monthly_data['Overall Month'] <= construction_term, 0, completed_value * (e**(home_appreciation_month * (monthly_data['Overall Month'] - construction_term))))

# Calculate loan value
monthly_data['Loan Value'] = np.where(monthly_data['Loan Month'] != 0, total_loan_amount * (((1 + loan_rate_monthly) ** (loan_term_months)) - ((1 + loan_rate_monthly)**(monthly_data['Loan Month'] - 1))) / ((1 + loan_rate_monthly)**(loan_term_months) - 1), total_loan_amount)
monthly_data['Loan Value'] = np.where(monthly_data['Overall Month'] > (construction_term + loan_term_months), 0, monthly_data['Loan Value'])

# Calculate disbursement payments in a linear fashion
monthly_data['Construction Disbursements'] = np.where(monthly_data['Overall Month'] <= construction_term, total_loan_amount / construction_term, 0)

# Calculate monthly payments
monthly_data['Mortgage Payment'] = np.where((monthly_data['Overall Month'] > construction_term) & (monthly_data['Overall Month'] < (construction_term + loan_term_months)), total_loan_amount * (loan_rate_monthly * ((1 + loan_rate_monthly)**loan_term_months)) / ((1 + loan_rate_monthly)**loan_term_months - 1), 0)

# Calculate interest payment
monthly_data['Interest Payment'] = np.where(monthly_data['Overall Month'] <= construction_term, monthly_data['Construction Disbursements'].cumsum() * loan_rate_monthly, monthly_data['Loan Value'] * loan_rate_monthly)
monthly_data['Cumulative Interest Payments'] = monthly_data['Interest Payment'].cumsum()

# Calculate principal payments
monthly_data['Principal Payment'] = np.where(monthly_data['Overall Month'] > construction_term, monthly_data['Mortgage Payment'] - monthly_data['Interest Payment'], 0)
monthly_data['Cumulative Principal Payments'] = monthly_data['Principal Payment'].cumsum()

# Calculate property taxes
monthly_data['Property Tax Payment'] = np.where(monthly_data['Overall Month'] > construction_term, (((monthly_data['Home Value'] * assessment_rate) * (mill_levy / 1000)) / 12), 0)
monthly_data['Cumulative Property Taxes'] = monthly_data['Property Tax Payment'].cumsum()

# Calculate maintenance costs
monthly_data['Maintenance Costs'] = np.where(monthly_data['Overall Month'] > construction_term, ((square_footage * maintenance_per_sqft) / 12), 0)
monthly_data['Cumulative Maintenance Costs'] = monthly_data['Maintenance Costs'].cumsum()

# Calculate HOA costs
monthly_data['HOA Costs'] = initial_hoa * (e**(inflation_monthly * (monthly_data['Overall Month'] - 1)))
monthly_data['Cumulative HOA Costs'] = monthly_data['HOA Costs'].cumsum()

# Calculate total home payments
monthly_data['Total Home Payments'] = monthly_data['Interest Payment'] + monthly_data['Principal Payment'] + monthly_data['Property Tax Payment'] + monthly_data['Maintenance Costs'] + monthly_data['HOA Costs']
monthly_data['Cumulative Home Payments'] = monthly_data['Total Home Payments'].cumsum() + down_payment + (closing_costs * total_loan_amount)

# Calculate equity metrics
monthly_data['Equity Ownership Proportion'] = 1- (monthly_data['Loan Value'] / total_loan_amount)
monthly_data['Equity Value'] = monthly_data['Equity Ownership Proportion'] * monthly_data['Home Value']
monthly_data['Equity Gain'] = monthly_data['Equity Value'] - monthly_data['Cumulative Home Payments']
monthly_data['Gains upon Sale'] = monthly_data['Home Value'] - monthly_data['Cumulative Home Payments'] - monthly_data['Loan Value']

# Calculate rent payments
monthly_data['Rent Payments'] = np.where(monthly_data['Overall Month'] == 1, (initial_rent + down_payment), initial_rent * (e**(rent_growth_rate_monthly * (monthly_data['Overall Month'] - 1))))
monthly_data['Cumulative Rent Payments'] = monthly_data['Rent Payments'].cumsum()

# Calculate rent investment value
monthly_data['Rent Investment'] = down_payment * (e**(investment_growth_rate_monthly * (monthly_data['Overall Month'] - 1)))

# Calculate when intersection points
monthly_data['Decision'] = np.where(monthly_data['Cumulative Rent Payments'] - monthly_data['Cumulative Home Payments'] > 0, 'Buy', 'Rent')

monthly_data

,Overall Month,Overall Year,Year Start,Construction Month,Loan Month,Home Value,Loan Value,Construction Disbursements,Mortgage Payment,Interest Payment,Cumulative Interest Payments,Principal Payment,Cumulative Principal Payments,Property Tax Payment,Cumulative Property Taxes,Maintenance Costs,Cumulative Maintenance Costs,HOA Costs,Cumulative HOA Costs,Total Home Payments,Cumulative Home Payments,Equity Ownership Proportion,Equity Value,Equity Gain,Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment,Decision
0,1,0,1,1,0,0.000000e+00,160000.0,13333.333333,0.0,44.444444,44.444444,0.0,0.000000,0.000000,0.000000,0.000,0.000,150.000000,150.000000,194.444444,48194.444444,0.0,0.000000e+00,-48194.444444,-208194.444444,41500.000000,41500.000000,40000.000000,Rent
1,2,0,0,2,0,0.000000e+00,160000.0,13333.333333,0.0,88.888889,133.333333,0.0,0.000000,0.000000,0.000000,0.000,0.000,150.250208,300.250208,239.139097,48433.583542,0.0,0.000000e+00,-48433.583542,-208433.583542,1502.502084,43002.502084,40200.500834,Rent
2,3,0,0,3,0,0.000000e+00,160000.0,13333.333333,0.0,133.333333,266.666667,0.0,0.000000,0.000000,0.000000,0.000,0.000,150.500834,450.751043,283.834168,48717.417709,0.0,0.000000e+00,-48717.417709,-208717.417709,1505.008343,44507.510427,40402.006683,Rent
3,4,0,0,4,0,0.000000e+00,160000.0,13333.333333,0.0,177.777778,444.444444,0.0,0.000000,0.000000,0.000000,0.000,0.000,150.751878,601.502921,328.529656,49045.947365,0.0,0.000000e+00,-49045.947365,-209045.947365,1507.518781,46015.029208,40604.522585,Rent
4,5,0,0,5,0,0.000000e+00,160000.0,13333.333333,0.0,222.222222,666.666667,0.0,0.000000,0.000000,0.000000,0.000,0.000,151.003341,752.506262,373.225563,49419.172928,0.0,0.000000e+00,-49419.172928,-209419.172928,1510.033408,47525.062616,40808.053601,Rent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,357,29,0,0,0,1.263047e+06,0.0,0.000000,0.0,0.000000,57026.190680,0.0,158287.097882,699.938644,128352.304151,109.375,37734.375,271.501762,73111.824664,1080.815405,502511.792375,1.0,1.263047e+06,760535.384468,760535.384468,2715.017615,771118.246637,237194.256744,Buy
357,358,29,0,0,0,1.268321e+06,0.0,0.000000,0.0,0.000000,57026.190680,0.0,158287.097882,702.861139,129055.165290,109.375,37843.750,271.954642,73383.779305,1084.190781,503595.983156,1.0,1.268321e+06,764724.869452,764724.869452,2719.546417,773837.793054,238383.197903,Buy
358,359,29,0,0,0,1.273617e+06,0.0,0.000000,0.0,0.000000,57026.190680,0.0,158287.097882,705.795837,129760.961127,109.375,37953.125,272.408277,73656.187583,1087.579114,504683.562271,1.0,1.273617e+06,768932.985594,768932.985594,2724.082774,776561.875828,239578.098655,Buy
359,360,29,0,0,0,1.278934e+06,0.0,0.000000,0.0,0.000000,57026.190680,0.0,158287.097882,708.742788,130469.703915,109.375,38062.500,272.862670,73929.050253,1090.980458,505774.542729,1.0,1.278934e+06,773159.811822,773159.811822,2728.626698,779290.502526,240778.988872,Buy


In [12]:
# Shrink set to year starts for simpler graphs
year_data = monthly_data.loc[monthly_data['Year Start'] == 1]

year_data

,Overall Month,Overall Year,Year Start,Construction Month,Loan Month,Home Value,Loan Value,Construction Disbursements,Mortgage Payment,Interest Payment,Cumulative Interest Payments,Principal Payment,Cumulative Principal Payments,Property Tax Payment,Cumulative Property Taxes,Maintenance Costs,Cumulative Maintenance Costs,HOA Costs,Cumulative HOA Costs,Total Home Payments,Cumulative Home Payments,Equity Ownership Proportion,Equity Value,Equity Gain,Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment,Decision
0,1,0,1,1,0,0.000000e+00,160000.000000,13333.333333,0.000000,44.444444,44.444444,0.000000,0.000000,0.000000,0.000000,0.000,0.000,150.000000,150.000000,194.444444,48194.444444,0.000000,0.000000e+00,-48194.444444,-208194.444444,41500.000000,41500.000000,40000.000000,Rent
12,13,1,1,0,1,3.012526e+05,160000.000000,0.000000,1183.500681,533.333333,4000.000000,650.167348,650.167348,166.944153,166.944153,109.375,109.375,153.030201,1969.636124,1612.850035,54896.122625,0.000000,0.000000e+00,-54896.122625,86356.485162,1530.302010,59696.361238,42473.461862,Buy
24,25,2,1,0,13,3.166982e+05,152053.353730,0.000000,1183.500681,506.844512,10228.873081,676.656169,8623.302438,175.503563,2225.485178,109.375,1421.875,156.121616,3826.031336,1624.500860,74325.567033,0.049667,1.572930e+04,-58596.265496,90319.238511,1561.216161,78260.313356,45099.874063,Buy
36,37,3,1,0,25,3.329356e+05,143782.948831,0.000000,1183.500681,479.276496,16132.908337,704.224185,16921.275354,184.501823,4389.569858,109.375,2734.375,159.275482,5719.928218,1636.652986,93898.056768,0.101357,3.374521e+04,-60152.844244,95254.615523,1592.754820,97199.282183,47888.694525,Buy
48,49,4,1,0,37,3.500056e+05,135175.594875,0.000000,1183.500681,450.585316,21698.871373,732.915365,25557.320490,193.961434,6664.609532,109.375,4046.875,162.493060,7652.084356,1649.330175,113619.760751,0.155153,5.430425e+04,-59315.506393,101210.239813,1624.930602,116520.843560,50849.966013,Buy
60,61,5,1,0,49,3.679508e+05,126217.564038,0.000000,1183.500681,420.725213,26912.988605,762.775468,34545.211429,203.906050,9056.292985,109.375,5359.375,165.775638,9623.272637,1662.557368,133497.140656,0.211140,7.768921e+04,-55807.933210,108236.061360,1657.756377,136232.726367,53994.352303,Buy
72,73,6,1,0,61,3.868160e+05,116894.569204,0.000000,1183.500681,389.648564,31760.925293,793.852117,43899.282913,214.360536,11570.600670,109.375,6671.875,169.124528,11634.281562,1676.360745,153536.965439,0.269409,1.042117e+05,-49325.274533,116384.470600,1691.245277,156342.815622,57333.176582,Buy
84,85,7,1,0,73,4.066485e+05,107191.741175,0.000000,1183.500681,357.305804,36227.762676,826.194877,53634.453702,225.351036,14213.819667,109.375,7984.375,172.541070,13685.915563,1690.767787,173746.326608,0.330052,1.342150e+05,-39531.336011,125710.418144,1725.410698,176859.155628,60878.462225,Buy
96,97,8,1,0,85,4.274978e+05,97093.604962,0.000000,1183.500681,323.645350,40297.974180,859.855331,63766.250369,236.905031,16992.559400,109.375,9296.875,176.026631,15778.995319,1705.807342,194132.654269,0.393165,1.680772e+05,-26055.495132,136271.540409,1760.266306,197789.953194,64642.976088,Buy
108,109,9,1,0,97,4.494161e+05,86584.055099,0.000000,1183.500681,288.613517,43955.400656,894.887164,74310.832065,249.051411,19913.768166,109.375,10609.375,179.582604,17914.358092,1721.509697,214703.733979,0.458850,2.062144e+05,-8489.320194,148128.291448,1795.826045,219143.580919,68640.274487,Buy


In [13]:
# Plot payment comparison
payments = figure(title='Rent vs. Buy Cumulative Cost')
payments.xaxis.axis_label = "Year"
payments.yaxis.formatter.use_scientific = False
payments.yaxis.axis_label = "Cost"
payments.line(year_data['Overall Year'], year_data['Cumulative Rent Payments'], line_color='red', legend_label='Renting')
payments.line(year_data['Overall Year'], year_data['Cumulative Home Payments'], line_color='blue', legend_label='Building')

show(payments)

In [14]:
# Plot returns comparison
returns = figure(title='Rent vs. Buy Cumulative Returns')
returns.xaxis.axis_label = "Year"
returns.yaxis.formatter.use_scientific = False
returns.yaxis.axis_label = "Returns"
returns.line(year_data['Overall Year'], year_data['Rent Investment'], line_color='red', legend_label='Renting')
returns.line(year_data['Overall Year'], year_data['Equity Gain'], line_color='blue', legend_label='Equity Gain')
returns.line(year_data['Overall Year'], year_data['Gains upon Sale'], line_color='green', legend_label='Gains upon Sale')
# Include line below if you wish to see the value of the home in relation to the equity value
# payments.line(year_data['Overall Year'], year_data['Home Value'], line_color='yellow', legend_label='Home Value')

show(returns)